<p align="center">
<img src="https://www.fasttrack.co.uk/wp-content/uploads/lendable.png" width="30%">
</p>
<br><br>


**In light of coronavirus, Lendable has taken the business decision to make manual income verification mandatory.**

This workbook shows how I used data to inform my thinking on:
* A/B randomised test strategy and process
* Cost-benefit analysis of the randomised test
* Potential improvements to the income validation process
* Further data desired to inform decision

Two Excel files named `loanapplications.csv` and `session and seed table.csv` have been used in the analysis
<br>

In [1]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# **Analysis of simplified merge test data**

In [2]:
# Import test
url_test = 'https://raw.githubusercontent.com/caryntan/sharing-github/master/session%20and%20seed%20table.csv'
df_test = pd.read_csv(url_test)

In [3]:
# Data dimensions, headers and number of unique values
print (df_test.shape)
df_test.head(5) 

(1010, 5)


,Session id,id,lifetime_id,name,value
0,000659de-7642-11ea-a8e5-069d8de3f4da,33352598,00051350-7642-11ea-a8e5-069d8de3f4da,mandatory-income-verification,mandatory
1,000a58c5-7b77-11ea-a8e5-069d8de3f4da,33641470,00091995-7b77-11ea-a8e5-069d8de3f4da,mandatory-income-verification,mandatory
2,00601bb1-7c05-11ea-a8e5-069d8de3f4da,33670653,005f2060-7c05-11ea-a8e5-069d8de3f4da,mandatory-income-verification,mandatory
3,0092b9c9-7a2c-11ea-a8e5-069d8de3f4da,33563434,00917986-7a2c-11ea-a8e5-069d8de3f4da,mandatory-income-verification,mandatory
4,00bf3b7f-7c1f-11ea-a8e5-069d8de3f4da,33679780,00be3938-7c1f-11ea-a8e5-069d8de3f4da,mandatory-income-verification,mandatory


In [4]:
# understand values 
df_test.nunique().to_frame().reset_index()

,index,0
0,Session id,1010
1,id,1010
2,lifetime_id,1010
3,name,1
4,value,2


In [5]:
# unique values in 'value' column
print(df_test['value'].unique())

['mandatory' 'normal']


In [6]:
# group by values
value = df_test.groupby('value')
df_test.groupby('value').value.agg('count').to_frame('count').reset_index()

,value,count
0,mandatory,780
1,normal,230


In [7]:
# randomised test split groups percentage
count = (230/(230+780))
percentage = "{:.0%}".format(count)
print(percentage)

23%


In [8]:
# session and seed data set has 1010 unique rows 
# 230 (23%) of those rows allow the applicant to go through the loan application process without manual income verfication 
# randomised A/B split test of 77% to 23% to test hypothesis of usefulness of manual income validation requirement during covid 

# **Analysis of loan applications data**

In [9]:
# Import test
url_loan = 'https://raw.githubusercontent.com/caryntan/sharing-github/master/loanapplications.csv'
df_loan = pd.read_csv(url_loan)

In [10]:
# Data dimensions, headers and number of unique values
print (df_loan.shape)
df_loan.head(5) 

(744, 10)


,Application id,Session id,Loan Amount,Loan Term,Loan issued,Loan missed first payment,Customer completed open banking,Customer uploaded a payslip,Customer signed contract,Customer is a homeowner
0,18808923,a6f66c9e-74c0-11ea-a8e5-069d8de3f4da,1000.0,12,0,0,1,0,0,0
1,18848975,71489d11-75da-11ea-a8e5-069d8de3f4da,7000.0,30,1,0,1,1,1,0
2,18849326,bd5ec027-75db-11ea-a8e5-069d8de3f4da,8000.0,60,1,0,1,0,1,1
3,18849472,452fda42-75dc-11ea-a8e5-069d8de3f4da,6000.0,30,1,0,1,0,1,0
4,18849611,c44b91c5-75dc-11ea-a8e5-069d8de3f4da,1500.0,24,0,0,0,0,0,0


In [11]:
# understand values 
df_loan.nunique().to_frame().reset_index()

,index,0
0,Application id,744
1,Session id,744
2,Loan Amount,83
3,Loan Term,9
4,Loan issued,2
5,Loan missed first payment,2
6,Customer completed open banking,2
7,Customer uploaded a payslip,2
8,Customer signed contract,2
9,Customer is a homeowner,2


In [12]:
# distribution of key numeric columns
round(df_loan[['Loan Amount','Loan Term']].describe())

,Loan Amount,Loan Term
count,744.0,744.0
mean,4021.0,34.0
std,3155.0,15.0
min,500.0,12.0
25%,1500.0,24.0
50%,3000.0,36.0
75%,5000.0,48.0
max,20000.0,60.0


In [13]:
#loan term to years
60/12

5.0

# **Merge tables**

In [22]:
# Merge 
df_test.drop(['id','lifetime_id','name'], axis = 1, inplace = True)
df_test.info()
df_combined = df_loan.merge(df_test, on="Session id", how = "left")
df_combined.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010 entries, 0 to 1009
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Session id  1010 non-null   object
 1   value       1010 non-null   object
dtypes: object(2)
memory usage: 15.9+ KB


,Application id,Session id,Loan Amount,Loan Term,Loan issued,Loan missed first payment,Customer completed open banking,Customer uploaded a payslip,Customer signed contract,Customer is a homeowner,value
0,18808923,a6f66c9e-74c0-11ea-a8e5-069d8de3f4da,1000.0,12,0,0,1,0,0,0,mandatory
1,18848975,71489d11-75da-11ea-a8e5-069d8de3f4da,7000.0,30,1,0,1,1,1,0,mandatory
2,18849326,bd5ec027-75db-11ea-a8e5-069d8de3f4da,8000.0,60,1,0,1,0,1,1,mandatory
3,18849472,452fda42-75dc-11ea-a8e5-069d8de3f4da,6000.0,30,1,0,1,0,1,0,mandatory
4,18849611,c44b91c5-75dc-11ea-a8e5-069d8de3f4da,1500.0,24,0,0,0,0,0,0,mandatory


In [23]:
#shape
print (df_combined.shape)

(744, 11)


In [24]:
# understand values 
df_combined.nunique().to_frame().reset_index()

,index,0
0,Application id,744
1,Session id,744
2,Loan Amount,83
3,Loan Term,9
4,Loan issued,2
5,Loan missed first payment,2
6,Customer completed open banking,2
7,Customer uploaded a payslip,2
8,Customer signed contract,2
9,Customer is a homeowner,2


In [25]:
# group by values
value = df_combined.groupby('value')
df_combined.groupby('value').value.agg('count').to_frame('count').reset_index()

,value,count
0,mandatory,574
1,normal,170


In [26]:
# randomised test split groups percentage
combinedcount = (170/(170+574))
combinedpercent = "{:.0%}".format(combinedcount)
print(combinedpercent)

23%


In [ ]:
# income val requirement combined
#customer completed open banking or customer uploaded a payslip

In [20]:
#first loan missed

In [ ]:
#open banking

In [ ]:
# payslip

In [21]:
# contract

In [ ]:
#homeowner

In [ ]:
#across the board, it seems like mandatory customers are better customers but would be good to understand whether they would have been a good customer anyway 